In [30]:
from __future__ import division
from __future__ import print_function

import tensorflow as tf

from models import GCN
import importlib
import utils
importlib.reload(utils)
from utils import *

# Set random seed
seed = 123
np.random.seed(seed)
tf.set_random_seed(seed)


In [ ]:
# get paths to available grahs metadata
N_sample = 200000
adj_mat, features_mat, train_labels_mat, test_labels_mat, val_labels_mat, train_mask_mat, test_mask_mat, val_mask_mat = load_data("txt_graph2216_21012020", N_sample)

In [ ]:
print(adj_mat.shape)
print(features_mat.shape)
print(train_labels_mat.shape)
print(test_labels_mat.shape)
print(val_labels_mat.shape)
print(train_mask_mat.shape)
print(test_mask_mat.shape)
print(val_mask_mat.shape)

print("number of masked elements")
print("train: " + str(len(np.where(train_mask_mat[:,1]==1)[0])))
print("test: " + str(len(np.where(test_mask_mat[:,1]==1)[0])))
print("val: " + str(len(np.where(val_mask_mat[:,1]==1)[0])))

print("\n positions of ones in masked labels")
print(np.where(train_labels_mat[np.array(train_mask_mat[:,1],dtype=bool)]==1)[1])
print(np.where(test_labels_mat[np.array(test_mask_mat[:,1],dtype=bool)]==1)[1])
print(np.where(val_labels_mat[np.array(val_mask_mat[:,1],dtype=bool)]==1)[1])

print("\n ones outside the mask")
print(np.where(train_labels_mat[np.logical_not(np.array(train_mask_mat[:,1],dtype=bool))]==1))
print(np.where(test_labels_mat[np.logical_not(np.array(test_mask_mat[:,1],dtype=bool))]==1))
print(np.where(val_labels_mat[np.logical_not(np.array(val_mask_mat[:,1],dtype=bool))]==1))

#sum_global_nodes = np.sum(adj_mat.toarray()[:,np.array(train_mask_mat[:,1],dtype=bool)], axis = 1) + np.sum(adj_mat.toarray()[:,np.array(test_mask_mat[:,1],dtype=bool)], axis = 1) + np.sum(adj_mat.toarray()[:,np.array(val_mask_mat[:,1],dtype=bool)], axis = 1)
#print(adj_mat.shape[1] - sum(sum_global_nodes))

In [ ]:
#delete all flags before declaration new one
del_all_flags(tf.flags.FLAGS)

# Settings
flags = tf.app.flags
FLAGS = tf.app.flags.FLAGS
tf.app.flags.DEFINE_string('mode', '', 'kernel') # for line by line mode
tf.app.flags.DEFINE_string('port', '', 'kernel') # for line by line mode
tf.app.flags.DEFINE_string('f', '', 'kernel') # for jupyter notebook

flags.DEFINE_float('learning_rate', 0.001, 'Initial learning rate.') #0.01
flags.DEFINE_integer('epochs', 2000, 'Number of epochs to train.') #200
flags.DEFINE_integer('hidden1', 64, 'Number of units in hidden layer 1.') #16
flags.DEFINE_float('dropout', 0.8, 'Dropout rate (1 - keep probability).') #0.5
flags.DEFINE_float('weight_decay', 5e-4, 'Weight for L2 loss on embedding matrix.')
flags.DEFINE_integer('early_stopping', 2000, 'Tolerance for early stopping (# of epochs).') #10

# Some preprocessing
features = preprocess_features(features_mat)
support = [preprocess_adj(adj_mat)]
num_supports = 1
model_func = GCN 

In [ ]:
# Define placeholders
placeholders = {
    'support': [tf.sparse_placeholder(tf.float32) for _ in range(num_supports)],
    'features': tf.sparse_placeholder(tf.float32, shape=tf.constant(features[2], dtype=tf.int64)),
    'labels': tf.placeholder(tf.float32, shape=(None, train_labels_mat.shape[1])),
    'labels_mask': tf.placeholder(tf.int32),
    'dropout': tf.placeholder_with_default(0., shape=()),
    'num_features_nonzero': tf.placeholder(tf.int32)  # helper variable for sparse dropout
}

# Create model
model = model_func(placeholders, input_dim=features[2][1], logging=True)

In [ ]:
# Initialize session
sess = tf.Session()
# Init variables
sess.run(tf.global_variables_initializer())

In [ ]:
# Train model
cost_val = []

for epoch in range(FLAGS.epochs):

    t = time.time()
    # Construct feed dictionary
    feed_dict = construct_feed_dict(features, support, train_labels_mat, train_mask_mat[:,1], placeholders)
    feed_dict.update({placeholders['dropout']: FLAGS.dropout})

    # Training step
    outs = sess.run([model.opt_op, model.loss, model.accuracy], feed_dict=feed_dict)

    # Validation
    cost, acc, duration = evaluate(features, support, val_mask_mat, val_mask_mat[:,1], placeholders, sess, model)
    cost_val.append(cost)

    # Print results
    print("Epoch:", '%04d' % (epoch + 1), "train_loss=", "{:.5f}".format(outs[1]),
          "train_acc=", "{:.5f}".format(outs[2]), "val_loss=", "{:.5f}".format(cost),
          "val_acc=", "{:.5f}".format(acc), "time=", "{:.5f}".format(time.time() - t))

    if epoch > FLAGS.early_stopping and cost_val[-1] > np.mean(cost_val[-(FLAGS.early_stopping+1):-1]):
        print("Early stopping...")
        break

print("Optimization Finished!")

In [ ]:
# Testing
test_cost, test_acc, test_duration = evaluate(features, support, test_labels_mat, test_mask_mat[:,1], placeholders, sess, model)
print("Test set results:", "cost=", "{:.5f}".format(test_cost),
      "accuracy=", "{:.5f}".format(test_acc), "time=", "{:.5f}".format(test_duration))